In [ ]:
import collections
import pandas as pd
import peppi_py
import melee

from slippi_db import upload_lib, test_peppi, preprocessing
from slippi_db.test_peppi_ray import TEST_KEY

In [ ]:
ENV = 'prod'

In [ ]:
prod_test_db = upload_lib.get_db(ENV, TEST_KEY)

In [ ]:
infos = list(prod_test_db.find({}))

In [ ]:
df = pd.DataFrame(infos)

In [ ]:
by_message = collections.defaultdict(list)
for info in infos:
    if not info['passed']:
        by_message[info['message']].append(info)

In [ ]:
%mkdir -p peppi_test

In [ ]:
local_files = {}
for message in by_message:
    info = by_message[message][0]
    path = upload_lib.download_slp_locally(ENV, info['key'], dir='peppi_test')
    local_files[message] = path

In [ ]:
print(local_files.keys())

In [ ]:
jumps_msg = "<class 'KeyError'>: 'jumps'"
event_61 = "<class 'ValueError'>: 61 is not a valid EventType"
char_27 = "<class 'ValueError'>: 27 is not a valid Character"

In [ ]:
%pdb

In [ ]:
local_files[char_27]

In [ ]:
char_27_game = peppi_py.game(local_files[char_27])

In [ ]:
char_27_game['metadata']

In [ ]:
preprocessing.assert_same_parse(local_files[value_61])

In [ ]:
value_61_game = peppi_py.game(local_files[value_61])

In [ ]:
value_61_game['start']

In [ ]:
# p0_action_path = local_files["AssertionError: ('p0', 'action')"]
p0_action_path = 'peppi_test/21_35_46 [NOAH] Ice Climbers + [TITP] Captain Falcon (FD).slp'

In [ ]:
game_rollbacks = peppi_py.game(p0_action_path, rollbacks=True)

In [ ]:
game_rollbacks['frames'].field('index')[:10]

In [ ]:
game = peppi_py.game(p0_action_path)

In [ ]:
# frames = game['frames']
frames = game_rollbacks['frames']
index = frames.field('index').to_numpy()
first_indices = []
next_idx = -123
for i, idx in enumerate(index):
    if idx == next_idx:
        first_indices.append(i)
        next_idx += 1

In [ ]:
print(len(frames), len(first_indices))

In [ ]:
def iter_libmelee(path: str):
    console = melee.Console(
        is_dolphin=False,
        allow_old_version=True,
        path=path)
    console.connect()
    
    while True:
        gamestate = console.step()
        if gamestate is None:
            break
        yield gamestate

In [ ]:
states = list(iter_libmelee(p0_action_path))

In [ ]:
len(states)

In [ ]:
first_frame_indices = [index[i] for i in first_indices]

In [ ]:
all_frame_indices = set(index)

In [ ]:
len(all_frame_indices)

In [ ]:
index

In [ ]:
raw_db = upload_lib.get_db(ENV, 'raw')
slp_db = upload_lib.get_db(ENV, 'slp')

def raw_from_slp_key(slp_key: str):
    slp_info = slp_db.find_one({'key': slp_key})
    raw_info = raw_db.find_one({'key': slp_info['raw_key']})
    return raw_info

In [ ]:
slp_key = by_message[event_61][0]['key']

In [ ]:
raw_info = raw_from_slp_key(slp_key)

In [ ]:
raw_info

In [ ]:
from slippi_db import parse_libmelee
from importlib import reload
reload(melee.console)

In [ ]:
parse_libmelee.get_slp('peppi_test/21_35_46 [NOAH] Ice Climbers + [TITP] Captain Falcon (FD).slp')